In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import date, datetime as dt
from scipy.special import factorial
import scipy.stats as scipy
from tqdm.notebook import trange

sns.set()

from nea_schema.esi.mkt import MarketHist
from nea_schema.sde.map import Region
from nea_schema.sde.inv import Type
from config.config import sql_params

In [ ]:
engine = create_engine('{engine}://{user}:{passwd}@{host}/{db}'.format(**sql_params))
Session = sessionmaker(bind=engine)
conn = Session()

In [ ]:
regions = pd.DataFrame([
    {'region_id': region.region_id, 'region_name': region.name.item_name}
    for region in conn.query(Region)
]).set_index('region_id')['region_name']
region_ids = regions.index[regions == 'Domain'].values.astype(str)

In [ ]:
types = pd.DataFrame([
    {'type_id': type.type_id, 'type_name': type.type_name}
    for type in conn.query(Type)
]).set_index('type_id')['type_name']
type_ids = types.loc[34:40].index.values.astype(str)

In [ ]:
start_date = '2018-12-31'
end_date = '2020-01-01'
result = conn.query(MarketHist)\
    .filter(MarketHist.region_id.in_(region_ids))\
    .filter(MarketHist.type_id.in_(type_ids))\
    .filter(MarketHist.record_date >= start_date)\
    .filter(MarketHist.record_date < end_date)
mkt_hist_raw = pd.read_sql(result.statement, result.session.bind)

In [ ]:
avg_hist = pd.DataFrame(index=pd.date_range(mkt_hist_raw['record_date'].min(), mkt_hist_raw['record_date'].max()))
vol_hist = pd.DataFrame(index=pd.date_range(mkt_hist_raw['record_date'].min(), mkt_hist_raw['record_date'].max()))
for grp, data in mkt_hist_raw.groupby('type_id'):
    avg_hist = avg_hist.join(data.set_index('record_date')['average'].rename(grp))
    vol_hist = vol_hist.join(data.set_index('record_date')['volume'].rename(grp))
avg_hist = avg_hist.fillna(method='ffill').fillna(method='bfill')
vol_hist = vol_hist.fillna(0)

In [ ]:
plt.figure(figsize=(12,4))
p = sns.lineplot(x=avg_hist.index, y=avg_hist[34].rename('Tritanium'))
_ = p.set(xlabel='Date', ylabel='Average Price', title='Average Transaction Price of Tritanium\nCalendar Year 2019')
p.set_xlim(dt(2019,1,1), dt(2020,1,1))
plt.tight_layout()
p.get_figure().savefig('avg_trit_cy2019.png')

In [ ]:
plt.figure(figsize=(12,4))
p = sns.lineplot(x=avg_hist.index, y=avg_hist[37].rename('Isogen'))
_ = p.set(xlabel='Date', ylabel='Average Price', title='Average Transaction Price of Isogen\nCalendar Year 2019')
p.set_xlim(dt(2019,1,1), dt(2020,1,1))
plt.tight_layout()
p.get_figure().savefig('avg_iso_cy2019.png')

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,8))

p1 = sns.lineplot(x=vol_hist.index, y=vol_hist[34].rename('Tritanium'), ax=ax[0])
_ = p1.set(xlabel='Date', ylabel='Volume', title='Volume Transacted of Tritanium\nCalendar Year 2019')
_ = p1.set_xlim(dt(2019,1,1), dt(2020,1,1))

p2 = sns.lineplot(x=vol_hist.index, y=vol_hist[37].rename('Isogen'), ax=ax[1])
_ = p2.set(xlabel='Date', ylabel='Volume', title='Volume Transacted of Isogen\nCalendar Year 2019')
_ = p2.set_xlim(dt(2019,1,1), dt(2020,1,1))

plt.tight_layout()
fig.savefig('vol_trit_iso_cy2019.png')

In [ ]:
log_returns = np.log(avg_hist / avg_hist.shift(1)).iloc[1:]

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,8))

p1 = sns.lineplot(x=log_returns.index, y=log_returns[34].rename('Tritanium'), ax=ax[0])
_ = p1.set(xlabel='Date', ylabel='Log Return', title='Log Returns of Tritanium\nCalendar Year 2019')
_ = p1.set_xlim(dt(2019,1,1), dt(2020,1,1))

p2 = sns.lineplot(x=log_returns.index, y=log_returns[37].rename('Isogen'), ax=ax[1])
_ = p2.set(xlabel='Date', ylabel='Log Return', title='Log Returns of Isogen\nCalendar Year 2019')
_ = p2.set_xlim(dt(2019,1,1), dt(2020,1,1))

plt.tight_layout()
fig.savefig('lnret_trit_iso_cy2019.png')

In [ ]:
log_returns.mean(axis=0)

In [ ]:
colors = pd.DataFrame([
    ['Tritanium', '#e6ab02'],
    ['Pyerite', '#d95f02'],
    ['Mexallon', '#66a61e'],
    ['Isogen', '#7570b3'],
    ['Nocxium', '#a6761d'],
    ['Zydrine', '#1b9e77'],
    ['Megacyte', '#e7298a'],
], index = np.arange(34,41), columns=['title', 'color'])

In [ ]:
vol_ratios = vol_hist / vol_hist.sum(axis=1).values[:,np.newaxis]

In [ ]:
vol_cum_ratios = vol_ratios.cumsum(axis=1)
vol_cum_ratios.insert(0, 'base', 0)
plt.figure(figsize=(12,8))
for i in range(1, vol_cum_ratios.columns.size):
    plt.fill_between(
        x=vol_cum_ratios.index,
        y1=vol_cum_ratios.iloc[:,i-1],
        y2=vol_cum_ratios.iloc[:,i],
        color=colors.iloc[i-1]['color'],
        label=colors.iloc[i-1]['title'],
    )
_ = plt.xlim(dt(2019,1,1), dt(2020,1,1))
_ = plt.ylim(0,1)
_ = plt.legend(loc=3)
_ = plt.xlabel('Date')
_ = plt.ylabel('Volume Proportion')
_ = plt.title('Core Minerals Volume Proportion by Day\nCalendar Year 2019')
plt.tight_layout()
plt.savefig('vol_prop_minerals_cy2019.png')

In [ ]:
vol_ratios.max(axis=0)

In [ ]:
value_hist = avg_hist * vol_hist

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,8))

p1 = sns.lineplot(x=value_hist.index, y=value_hist[34].rename('Tritanium'), ax=ax[0])
_ = p1.set(xlabel='Date', ylabel='Value', title='Market Value of Tritanium\nCalendar Year 2019')
_ = p1.set_xlim(dt(2019,1,1), dt(2020,1,1))

p2 = sns.lineplot(x=value_hist.index, y=value_hist[37].rename('Isogen'), ax=ax[1])
_ = p2.set(xlabel='Date', ylabel='Value', title='Market Value of Isogen\nCalendar Year 2019')
_ = p2.set_xlim(dt(2019,1,1), dt(2020,1,1))

plt.tight_layout()
fig.savefig('value_trit_iso_cy2019.png')

In [ ]:
value_ratios = value_hist / value_hist.sum(axis=1).values[:,np.newaxis]
value_cum_ratios = value_ratios.cumsum(axis=1)
value_cum_ratios.insert(0, 'base', 0)
plt.figure(figsize=(12,8))
for i in range(1, value_cum_ratios.columns.size):
    plt.fill_between(
        x=value_cum_ratios.index,
        y1=value_cum_ratios.iloc[:,i-1],
        y2=value_cum_ratios.iloc[:,i],
        color=colors.iloc[i-1]['color'],
        label=colors.iloc[i-1]['title'],
    )
_ = plt.xlim(dt(2019,1,1), dt(2020,1,1))
_ = plt.ylim(0,1)
_ = plt.legend(loc=3)
_ = plt.xlabel('Date')
_ = plt.ylabel('Value Proportion')
_ = plt.title('Core Minerals Market Value Proportion by Day\nCalendar Year 2019')
plt.tight_layout()
plt.savefig('value_prop_minerals_cy2019.png')

In [ ]:
value_ratios.mean(axis=0)

In [ ]:
cum_log_returns = log_returns.cumsum(axis=0)

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,8))

p1 = sns.lineplot(x=cum_log_returns.index, y=cum_log_returns[34].rename('Tritanium'), ax=ax[0])
_ = p1.set(xlabel='Date', ylabel='Cumulative Log Returns', title='Cumulative Log Returns of Tritanium\nCalendar Year 2019')
_ = p1.set_xlim(dt(2019,1,1), dt(2020,1,1))

p2 = sns.lineplot(x=cum_log_returns.index, y=cum_log_returns[37].rename('Isogen'), ax=ax[1])
_ = p2.set(xlabel='Date', ylabel='Cumulative Log Returns', title='Cumulative Log Returns of Isogen\nCalendar Year 2019')
_ = p2.set_xlim(dt(2019,1,1), dt(2020,1,1))

plt.tight_layout()
fig.savefig('cumlogret_trit_iso_cy2019.png')

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0, color='#000000')
for idx, color_data in colors.iterrows():
    sns.lineplot(x=cum_log_returns.index, y=cum_log_returns[idx].rename(color_data['title']), label=color_data['title'], color=color_data['color'])

_ = plt.xlim(dt(2019,1,1), dt(2020,1,1))
_ = plt.legend(loc=3)
_ = plt.xlabel('Date')
_ = plt.ylabel('Cumulative Log Returns')
_ = plt.title('Core Minerals Market Cumulative Log Returns by Day\nFrom January 1, 2019, for Calendar Year 2019')
plt.tight_layout()
plt.savefig('cumlogret_minerals_cy2019.png')

In [ ]:
index_value = (cum_log_returns * value_ratios.iloc[1:]).sum(axis=1)

In [ ]:
plt.figure(figsize=(12,8))
p = sns.lineplot(x=index_value.index, y=index_value.rename('Core Minerals Index'))
_ = p.set(xlabel='Date', ylabel='Index Value', title='Core Minerals Index\nCalendar Year 2019')
p.set_xlim(dt(2019,1,1), dt(2020,1,1))
plt.tight_layout()
p.get_figure().savefig('minerals_index_cy2019.png')